In [1]:
import csv
import psycopg2
from psycopg2 import sql

In [2]:
data_path = '../../../data/raw/l_amat_20240501/EN.dat'

In [3]:
db_params = {
    'dbname': 'ross_projects',
    'user': 'rwardrup',
    'password': 'Rward0232',
    'host': '192.168.3.231',
    'options': '-c client_encoding=utf8'
}

In [4]:
drop_table_query = """
DROP TABLE IF EXISTS ham_ops.entity_data"""

create_table_query = """
CREATE TABLE if not exists ham_ops.entity_data (
    record_type CHAR(2),
    unique_system_identifier NUMERIC(9,0),
    uls_file_number CHAR(14),
    ebf_number VARCHAR(30),
    call_sign CHAR(10),
    entity_type CHAR(2),
    licensee_id CHAR(9),
    entity_name VARCHAR(200),
    first_name VARCHAR(40),
    mi CHAR(1),
    last_name VARCHAR(40),
    suffix CHAR(3),
    phone CHAR(10),
    fax CHAR(10),
    email VARCHAR(50),
    street_address VARCHAR(60),
    city VARCHAR(40),
    state CHAR(2),
    zip_code CHAR(9),
    po_box VARCHAR(40),
    attention_line VARCHAR(35),
    sgin CHAR(3),
    fcc_registration_number CHAR(10),
    applicant_type_code CHAR(1),
    applicant_type_code_other VARCHAR(40),
    status_code CHAR(1),
    status_date DATE,
    three_point_seven_ghz_license_type CHAR(1),
    linked_unique_system_identifier NUMERIC(9,0),
    linked_call_sign CHAR(10)
);
"""

In [ ]:
def setup_database(params, query):
    conn = psycopg2.connect(**params)
    conn.set_client_encoding('UTF8')
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()
    conn.close()

# Function to insert data into the database
def insert_data(params, data):
    conn = psycopg2.connect(**params)
    cursor = conn.cursor()
    insert_query = sql.SQL("""
    INSERT INTO ham_ops.entity_data VALUES ({})
    """).format(sql.SQL(', ').join(sql.Placeholder() * len(data[0])))
    cursor.executemany(insert_query, data)
    conn.commit()
    cursor.close()
    conn.close()

def process_and_load(filename, db_params):
    with open(filename, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='|')
        data = [row for row in reader if any(field.strip() for field in row)]

    if not data:
        print("No data to insert.")
        return

    insert_data(db_params, data)

In [ ]:
# Drop table
setup_database(db_params, drop_table_query)
# Set up the database table
setup_database(db_params, create_table_query)

# Process the .dat file and load the data into the database
process_and_load(data_path, db_params)